## Install ultralytics

In [124]:
!pip install ultralytics

In [125]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random
import os
import cv2
import shutil
import tqdm
import glob

In [126]:
# Check GPU is working or not
import torch
print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Using torch 2.5.1+cpu (CPU)


In [127]:
images_path = r'C:/Users/senne/github/background-removal-model/data/archive/colorful_fashion_dataset_for_object_detection/JPEGImages/'
annotations_path  = r'C:/Users/senne/github/background-removal-model/data/archive/colorful_fashion_dataset_for_object_detection/Annotations_txt/'
path = r'C:/Users/senne/github/background-removal-model/data/archive/colorful_fashion_dataset_for_object_detection/'

# Try YOLOv8

In [128]:
from ultralytics import YOLO

# load pre-trained model
detection_model = YOLO("yolov8m.pt")

# Prepare Dataset

In [129]:
trainval_path = r"C:/Users/senne/github/background-removal-model/data/archive/colorful_fashion_dataset_for_object_detection/ImageSets/Main/trainval.txt"
test_path = r"C:/Users/senne/github/background-removal-model/data/archive/colorful_fashion_dataset_for_object_detection/ImageSets/Main/test.txt"

train = []
with open(trainval_path, 'r') as f:
    for line in f.readlines():
        train.append(line.strip())

test = []
with open(test_path, 'r') as f:
    for line in f.readlines():
        test.append(line.strip())


In [130]:
len(train), len(test)

(2145, 537)

os.mkdir('train')
os.mkdir('train/images')
os.mkdir('train/labels')

os.mkdir('test')
os.mkdir('test/images')
os.mkdir('test/labels')

In [131]:
os.mkdir('train')
os.mkdir('train/images')
os.mkdir('train/labels')

os.mkdir('test')
os.mkdir('test/images')
os.mkdir('test/labels')

In [132]:
import shutil
from tqdm import tqdm

In [133]:
train_path = 'C:/Users/senne/github/background-removal-model/train/'
test_path = 'C:/Users/senne/github/background-removal-model/test/'

print('Copying Train Data..!!')
for i in tqdm(train):
    a = shutil.copyfile(images_path+i+'.jpg', train_path+'images/'+i+'.jpg')
    a = shutil.copyfile(annotations_path+i+'.txt', train_path+'labels/'+i+'.txt')

print('Copying Test Data..!!')
for i in tqdm(test):
    a = shutil.copyfile(images_path+i+'.jpg', test_path+'images/'+i+'.jpg')
    a = shutil.copyfile(annotations_path+i+'.txt', test_path+'labels/'+i+'.txt')

Copying Train Data..!!


100%|██████████| 2145/2145 [00:01<00:00, 1418.37it/s]


Copying Test Data..!!


100%|██████████| 537/537 [00:00<00:00, 1443.28it/s]


In [149]:
text = """
train: ./train/images
val: ./test/images

# number of classes
nc: 10

# class names
names: ['sunglass','hat','jacket','shirt','pants','shorts','skirt','dress','bag','shoe']
"""
with open("data.yaml", 'w') as file:
    file.write(text)

In [143]:
import os

base_path = "C:/Users/senne/github/background-removal-model/train"
required_dirs = ['images', 'labels']

for dir_name in required_dirs:
    dir_path = os.path.join(base_path, dir_name)
    if not os.path.exists(dir_path):
        print(f"Directory not found: {dir_path}")
    else:
        print(f"Directory exists: {dir_path}")

# Check if images and labels are matching
images = set(f.split('.')[0] for f in os.listdir(os.path.join(base_path, 'images')))
labels = set(f.split('.')[0] for f in os.listdir(os.path.join(base_path, 'labels')))
missing_labels = images - labels
if missing_labels:
    print(f"Missing labels for images: {missing_labels}")


Directory exists: C:/Users/senne/github/background-removal-model/train\images
Directory exists: C:/Users/senne/github/background-removal-model/train\labels


In [146]:
import os

# Paths
train_path = "C:/Users/senne/github/background-removal-model/train/images"
val_path = "C:/Users/senne/github/background-removal-model/test/images"

# Check paths
print(f"Train path exists: {os.path.exists(train_path)}")
print(f"Val path exists: {os.path.exists(val_path)}")


Train path exists: True
Val path exists: True


In [151]:
from ultralytics import YOLO

# Load the model
model = YOLO("yolov8m.pt")

# Train the model
model.train(data='data.yaml', epochs=5)


Ultralytics 8.3.49  Python-3.11.3 torch-2.5.1+cpu CPU (AMD Ryzen 5 7600X 6-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

train: Scanning C:\Users\senne\github\background-removal-model\datasets\train\labels... 2145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2145/2145 [00:00<00:00, 2204.59it/s]


train: New cache created: C:\Users\senne\github\background-removal-model\datasets\train\labels.cache


val: Scanning C:\Users\senne\github\background-removal-model\datasets\test\labels... 537 images, 0 backgrounds, 0 corrupt: 100%|██████████| 537/537 [00:00<00:00, 2087.85it/s]


val: New cache created: C:\Users\senne\github\background-removal-model\datasets\test\labels.cache
Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train3
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        537       2035      0.763      0.763      0.779      0.525



5 epochs completed in 2.038 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 52.0MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.49  Python-3.11.3 torch-2.5.1+cpu CPU (AMD Ryzen 5 7600X 6-Core Processor)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [01:17<00:00,  4.53s/it]


                   all        537       2035      0.762      0.763      0.779      0.525
              sunglass         82         82      0.931      0.166      0.367      0.125
                   hat         77         77      0.705      0.792      0.801      0.485
                jacket        179        181      0.736       0.84      0.867      0.662
                 shirt        361        366      0.798      0.844       0.85      0.598
                 pants        114        114      0.796      0.947      0.959      0.747
                shorts        107        107      0.789      0.839      0.792      0.523
                 skirt        186        186      0.759      0.817      0.804      0.622
                 dress        128        128      0.608      0.844      0.758      0.572
                   bag        274        274      0.711      0.712      0.767      0.437
                  shoe        520        520       0.79      0.831      0.826       0.48
Speed: 1.3ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022FA5E907D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,   

In [152]:
# Export the trained model
model.export(format='onnx')  # Change 'onnx' to your desired format


Ultralytics 8.3.49  Python-3.11.3 torch-2.5.1+cpu CPU (AMD Ryzen 5 7600X 6-Core Processor)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from 'runs\detect\train3\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 14, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.5/14.5 MB 101.2 MB/s eta 0:00:00

requirements: AutoUpdate success  8.7s, installed 2 packages: ['onnx>=1.12.0', 'onnxslim']
requirements:  Restart runtime or rerun command for updates to take effect

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.43...
ONNX: export success  10.9s, saved as 'runs\detect\train3\weights\best.onnx' (98.8 MB)

Export complete (11.6s)
Results saved to C:\Users\senne\github\background-removal-model\runs\detect\train3\weights
Predict:         yolo predict task=de

'runs\\detect\\train3\\weights\\best.onnx'

In [158]:
from ultralytics import YOLO

# Load the model
model = YOLO("runs/detect/train3/weights/best.pt")  # Adjust the path to your model file

# Perform inference on an image
results = model("uploads/1730629678.jpeg")  # Perform inference

# Iterate over the results and display/save each one
for result in results:  # `results` is a list of detections
    result.show()  # Display result
    result.save(save_dir="inference_results/")  # Save annotated image


image 1/1 C:\Users\senne\github\background-removal-model\uploads\1730629678.jpeg: 640x480 1 pants, 126.4ms
Speed: 2.0ms preprocess, 126.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


TypeError: Results.plot() got an unexpected keyword argument 'save_dir'